## Import des librairies

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt

## On met maintenant le tout dans un programme

In [5]:
#On crée des listes vides que l'on va remplir au fur et à mesure du scraping
employer = []
name_job = []
job_links = []
salaires = []
locations = []
types = []

base_url = 'https://www.reed.co.uk/jobs?sortBy=displayDate&pageno='
prefix = "https://www.reed.co.uk"
# Liste pour stocker les URL
urls = ["https://www.reed.co.uk/jobs?sortBy=displayDate&pageno=1"]

# Génération des URL de chaque page du site

for i in range(2, 6000):
    url = f'{base_url}{i}'
    urls.append(url)



for url in urls :
    #Extraction des informations du HTML
    response = requests.get(url)
    soup = BeautifulSoup(response.text)


    #Les noms des employeurs sont dans la classe "gtmJobListingPostedBy" donc on les extrait grâce à soup
    #et on les ajoute à la liste employer
    jobcreator_elements = soup.find_all(class_ = "gtmJobListingPostedBy")
    for jobcreator_element in jobcreator_elements :
        name_jobcreator = jobcreator_element.get_text()
        employer.append(name_jobcreator)

    #L'intitulé de l'emploi et le lien de l'emploi sont dans la classe "job-card_jobResultHeading__title__IQ8iT",
    #on les extrait et on les ajoutes à leur liste respective
    namejob_elements = soup.find_all(class_ = "job-card_jobResultHeading__title__IQ8iT")
    for namejob_element in namejob_elements :
        #On récupère l'initulé de l'emploi
        name_fonction = namejob_element.find('a').get_text()
        name_job.append(name_fonction)
        #On récupère le lien vers le site
        job_link = namejob_element.find('a')['href']
        job_link = prefix + job_link
        job_links.append(job_link)

    #Le salaire, la localisation et le type d'emploi sont renseignés dans la
    #classe 'job-card_jobMetadata__gjkG3 list-group list-group-horizontal'
    ul_elements = soup.find_all('ul', class_='job-card_jobMetadata__gjkG3 list-group list-group-horizontal')
    for ul in ul_elements:
        # Extraction des éléments <li>
        li_elements = ul.find_all('li')

        # Ajout des éléments dans les listes correspondantes (avec gestion du cas vide)
        salaires.append(li_elements[0].get_text() if li_elements and len(li_elements) > 0 else '0')
        locations.append(li_elements[1].get_text() if li_elements and len(li_elements) > 1 else '0')
        types.append(li_elements[2].get_text() if li_elements and len(li_elements) > 2 else '0')


#Ensuite, il ne reste plus qu'à créer le dataframe en fonction des listes choisies
index = [i for i in range (len(salaires))]
df = pd.DataFrame({"type" : types , "salary" :  salaires, "location" : locations, "employer" : employer,
                      "name_job" : name_job , "lien" : job_links }, index)

df

,type,salary,location,employer,name_job,lien
0,"Permanent, full-time",£12.33 - £13.33 per hour,"Birstall, Batley, West Yorkshire",Costa Coffee,Barista Maestro Team leader/ Supervisor,https://www.reed.co.uk/jobs/barista-maestro-te...
1,"Permanent, full-time","£33,000 - £36,000 per annum","East Horsley, Surrey",Revolution Bars,Deputy Manager,https://www.reed.co.uk/jobs/deputy-manager/534...
2,"Permanent, full-time","£30,000 - £32,000 per annum","Portsmouth, Hampshire",Workshop Recruitment,Finance and Accounts Administrator,https://www.reed.co.uk/jobs/finance-and-accoun...
3,"Permanent, full-time",Salary negotiable,"Huddersfield, West Yorkshire",Search,Payroll Manager,https://www.reed.co.uk/jobs/payroll-manager/53...
4,"Contract, full-time","£28,000 - £30,000 per annum","South West London, London",Language Matters,German speaking Customer Service Executive - 3...,https://www.reed.co.uk/jobs/german-speaking-cu...
5,"Permanent, full-time","£60,000 - £90,000 per annum",London,Harrington Boyd,SAP XI/PI Consultant,https://www.reed.co.uk/jobs/sap-xi-pi-consulta...
6,"Permanent, full-time",£12 per hour,"Havant, Hampshire",Workshop Recruitment,Customer Service Coordinator : Repairs and Mai...,https://www.reed.co.uk/jobs/customer-service-c...
7,"Permanent, full-time or part-time",£12 per hour,"Addlestone, Surrey",SKILLFRAME,Full or Part Time Shipping & Printing Associate,https://www.reed.co.uk/jobs/full-or-part-time-...
8,"Permanent, full-time","£35,000 - £40,000 per annum","Oswestry, Shropshire",Ernest Gordon Recruitment Limited,Production Manager Events,https://www.reed.co.uk/jobs/production-manager...
9,"Permanent, full-time","£28,000 - £32,000 per annum","Longbridge, Warwick, Warwickshire",SF Recruitment,Payroll Controller,https://www.reed.co.uk/jobs/payroll-controller...


On peut ensuite faire ce que l'on veut avec ce jeu de données, le nettoyer, créer un modèle de régression linéaire, etc.